### **Making sure to make COVID history section exactly the same in SHU to mimic HU format to help with data extraction.**

In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
from google.colab import files

# Step 2: Upload the Excel file
print("Please upload the Excel file:")
uploaded = files.upload()

# Step 3: Extract the uploaded file name
input_file_name = list(uploaded.keys())[0]

# Step 4: Load the Excel file into a DataFrame
shu_df = pd.read_excel(input_file_name)

# Step 5: Define the mapping
covid_mapping = {
    "I had/have COVID and tested positive": "I have/had COVID and was/am tested positive",
    "I think I had/have COVID, but did not get tested": "I think I have/had COVID, but did not get the test",
    "No, have not had COVID yet": "No, I have not contracted COVID yet"
}

# Step 6: Update the "COVID history?" column using the mapping
shu_df["COVID history?"] = shu_df["COVID history?"].map(covid_mapping)

# Step 7: Save the updated DataFrame to a new Excel file
output_file_name = "Updated_COVID_History.xlsx"
shu_df.to_excel(output_file_name, index=False)

# Step 8: Download the updated file
files.download(output_file_name)

print("Processing complete! The updated file is ready for download.")


Please upload the Excel file:


Saving SHU - Adolescent Health(1-184)ORIGNAL.xlsx to SHU - Adolescent Health(1-184)ORIGNAL.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing complete! The updated file is ready for download.


### **Merging all data and removing NaN values Hu+SHU**

In [ ]:
# Import necessary libraries
import pandas as pd
from google.colab import files

# Upload files in Colab
uploaded = files.upload()

# Load the uploaded Excel files
file_names = list(uploaded.keys())
df1 = pd.read_excel(file_names[0])
df2 = pd.read_excel(file_names[1])

# Remove blank columns
df1_cleaned = df1.dropna(axis=1, how="all")
df2_cleaned = df2.dropna(axis=1, how="all")

# Add respective ID columns to distinguish datasets
df1_cleaned.rename(columns={"ID": "HU_ID"}, inplace=True)
df2_cleaned.rename(columns={"ID": "SHU_ID"}, inplace=True)

# Identify common and unique fields
common_columns = set(df1_cleaned.columns).intersection(set(df2_cleaned.columns))
unique_to_df1 = set(df1_cleaned.columns) - common_columns
unique_to_df2 = set(df2_cleaned.columns) - common_columns

# Align rows for merging
df1_cleaned["Source"] = "HU"  # Mark source
df2_cleaned["Source"] = "SHU"  # Mark source

# Combine common columns
common_df = pd.concat([df1_cleaned[list(common_columns)], df2_cleaned[list(common_columns)]], ignore_index=True)

# Include unique columns, adding NaN where necessary
unique_df1 = df1_cleaned[list(unique_to_df1)].reindex(index=common_df.index, fill_value=None)
unique_df2 = df2_cleaned[list(unique_to_df2)].reindex(index=common_df.index, fill_value=None)

# Combine everything into one dataframe
merged_df = pd.concat([common_df, unique_df1, unique_df2], axis=1)

# Reorder columns to place IDs first
id_columns = ["HU_ID", "SHU_ID"]
other_columns = [col for col in merged_df.columns if col not in id_columns]
merged_df = merged_df[id_columns + other_columns]

# Save the merged file
output_file = "final_merged_dataset_dd_mm_yy.xlsx"
merged_df.to_excel(output_file, index=False)
files.download(output_file)


Saving HU - Student Health and Behaviour(1-61)ORIGNAL.xlsx to HU - Student Health and Behaviour(1-61)ORIGNAL (6).xlsx
Saving SHU - Adolescent Health(1-184)ORIGNAL.xlsx to SHU - Adolescent Health(1-184)ORIGNAL (7).xlsx


<ipython-input-8-8021e2c4697a>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_cleaned.rename(columns={"ID": "HU_ID"}, inplace=True)
<ipython-input-8-8021e2c4697a>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_cleaned.rename(columns={"ID": "SHU_ID"}, inplace=True)
<ipython-input-8-8021e2c4697a>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_cleaned["Source"] 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Normalizing and converting all weight data in kgs and all heights in cm.
*Excel file processed in previous code with some additions is used here*

In [ ]:
# Install required libraries
!pip install pandas openpyxl xlsxwriter

# Import necessary modules
import pandas as pd
import re
import numpy as np
from google.colab import files

# Step 1: Upload the file
print("Please upload your Excel file:")
uploaded = files.upload()

# Step 2: Input file name
file_name = list(uploaded.keys())[0]  # Automatically get the uploaded file name
print(f"Processing file: {file_name}")

# Step 3: Load the file
df = pd.read_excel(file_name)

# Function to clean and convert height to cm
def convert_height_to_cm(height):
    if pd.isnull(height) or not isinstance(height, str):
        return np.nan  # Treat missing or non-string entries as NaN

    # Handle cm directly
    cm_match = re.search(r'(\d+(\.\d+)?)\s*cm', height)
    if cm_match:
        return float(cm_match.group(1))

    # Handle meters
    m_match = re.search(r'(\d+(\.\d+)?)\s*m', height)
    if m_match:
        return float(m_match.group(1)) * 100

    # Handle feet and inches
    ft_in_match = re.search(r"(\d+)\s*(?:ft|'|feet)\s*(\d+)?\s*(?:in|\"|inches)?", height)
    if ft_in_match:
        feet = int(ft_in_match.group(1))
        inches = int(ft_in_match.group(2)) if ft_in_match.group(2) else 0
        return feet * 30.48 + inches * 2.54

    # Handle inches only
    in_match = re.search(r'(\d+)\s*(?:in|inches)', height)
    if in_match:
        return int(in_match.group(1)) * 2.54

    # Handle ranges
    range_match = re.search(r"(\d+)\s*ft\s*(\d+)-(\d+)\s*in", height)
    if range_match:
        feet = int(range_match.group(1))
        inch_min = int(range_match.group(2))
        inch_max = int(range_match.group(3))
        avg_inches = (inch_min + inch_max) / 2
        return feet * 30.48 + avg_inches * 2.54

    # Unconvertible entries
    return np.nan

# Function to clean and normalize weight to kg
def convert_weight_to_kg(weight):
    if pd.isnull(weight) or not isinstance(weight, str):
        return np.nan  # Treat missing or non-string entries as NaN

    # Handle kg directly
    kg_match = re.search(r'(\d+(\.\d+)?)\s*kg', weight)
    if kg_match:
        return float(kg_match.group(1))

    # Handle lbs (convert to kg)
    lbs_match = re.search(r'(\d+(\.\d+)?)\s*lbs?', weight)
    if lbs_match:
        lbs = float(lbs_match.group(1))
        return lbs * 0.453592  # Convert lbs to kg

    # Handle ranges (e.g., "120-125 lbs")
    range_match = re.search(r"(\d+(\.\d+)?)\s*-\s*(\d+(\.\d+)?)\s*(lbs|kg)", weight)
    if range_match:
        low = float(range_match.group(1))
        high = float(range_match.group(3))
        avg = (low + high) / 2
        if "lbs" in range_match.group(5).lower():
            avg *= 0.453592  # Convert lbs to kg
        return avg

    # Unconvertible entries
    return np.nan

# Apply height and weight conversions
df['Height_converted'] = df['Height'].apply(convert_height_to_cm)
df['Weight_converted'] = df['Weight'].apply(convert_weight_to_kg)

# Highlight outliers directly in the same columns
df['Height_final'] = df['Height'].where(df['Height_converted'].notna(), other="OUTLIER")
df['Weight_final'] = df['Weight'].where(df['Weight_converted'].notna(), other="OUTLIER")

# Save cleaned data with highlighting
cleaned_file_name = 'cleaned_data_with_highlights.xlsx'

# Write to Excel with formatting
with pd.ExcelWriter(cleaned_file_name, engine='xlsxwriter') as writer:
    df.to_excel(writer, index=False, sheet_name='Cleaned Data')
    workbook = writer.book
    worksheet = writer.sheets['Cleaned Data']

    # Define formats
    yellow_format = workbook.add_format({'bg_color': 'yellow'})

    # Apply conditional formatting for outliers in Height_final
    height_col_index = df.columns.get_loc('Height_final') + 1
    worksheet.conditional_format(1, height_col_index, len(df), height_col_index,
                                 {'type': 'text', 'criteria': 'containing', 'value': 'OUTLIER', 'format': yellow_format})

    # Apply conditional formatting for outliers in Weight_final
    weight_col_index = df.columns.get_loc('Weight_final') + 1
    worksheet.conditional_format(1, weight_col_index, len(df), weight_col_index,
                                 {'type': 'text', 'criteria': 'containing', 'value': 'OUTLIER', 'format': yellow_format})

print(f"Cleaned data saved to {cleaned_file_name}")

# Step 4: Download the cleaned file
files.download(cleaned_file_name)


Please upload your Excel file:


Saving final_merged_dataset_dd_mm_yy (Autosaved).xlsx to final_merged_dataset_dd_mm_yy (Autosaved) (1).xlsx
Processing file: final_merged_dataset_dd_mm_yy (Autosaved) (1).xlsx
Cleaned data saved to cleaned_data_with_highlights.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>